In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [110]:
from datetime import datetime

from jellyfish import utils
from jellyfish.history_loader import load_candles_history
from jellyfish.core import Client
from jellyfish.transform import sampling

utils.disable_warnings()
frame = load_candles_history(Client(), 'btcusdt', end_dt=datetime(year=2021, month=4, day=25), interval='1m', candles_num=5000 * 10)
original_frame = frame.copy()
# frame = sampling.volume_bars(frame[-5000:].reset_index(), 100).set_index('Date')

  0%|          | 0/50 [00:00<?, ?it/s]

In [104]:
from jellyfish.core import Strategy, Backtest
from jellyfish import indicator


class PlotIndicators(Strategy):
    def init(self):
        self.I(indicator.cmo, self.data.Close, 10)

bt = Backtest(frame, PlotIndicators)
bt.run()
bt.plot(open_browser=True)

Row(id='71718', ...)

In [130]:
from jellyfish.transform.sampling import _generic_sampling, DEFAULT_SAMPLING_AGG

frame['i_CMO'] = indicator.cmo(frame.Close, 10)
condition = lambda ohlc: ohlc['i_CMO'].max() > 0
new_frame = _generic_sampling(frame.reset_index()[10000:15000], condition, DEFAULT_SAMPLING_AGG).set_index('Date')
new_frame.High = new_frame.Low = new_frame.Close

utils.plot_ohlc(new_frame, open_browser=True)

  0%|          | 0/5000 [00:00<?, ?it/s]

Row(id='86515', ...)

In [138]:
from jellyfish.core import SmaCross

bt = Backtest(new_frame, SmaCross)
bt.run(
    n1=16,
    n2=37
)
bt.plot(open_browser=True)

Row(id='88573', ...)

In [147]:
from tqdm.notebook import trange

class ZlemaCross(SmaCross):
    def init(self):
        self.sma1 = self.I(indicator.zlema, self.data.Close, self.n1)
        self.sma2 = self.I(indicator.zlema, self.data.Close, self.n2)

# frame = original_frame.copy()
# frame['i_CMO'] = indicator.cmo(frame.Close, 10)
# condition = lambda ohlc: ohlc['i_CMO'].max() > 0
# frame = _generic_sampling(frame.reset_index(), condition, DEFAULT_SAMPLING_AGG).set_index('Date')

train_size = 1000
val_size = train_size // 2
stats = []
for mid in trange(train_size, len(frame), val_size // 2):
    bt = Backtest(frame[mid - train_size:mid], ZlemaCross, trade_on_close=True)
    bt.optimize(
        n1=range(3, 30),
        n2=range(2, 50, 5),
        constraint=lambda p: p.n1 < p.n2
    )
    params: SmaCross = bt._strategy

    bt = Backtest(frame[mid:mid + val_size], ZlemaCross, trade_on_close=True)
    stats.append(bt.run(
        n1=params.n1,
        n2=params.n2
    ))
    bt.plot(open_browser=True)

  0%|          | 0/95 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

...

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/13 [00:00<?, ?it/s]

Process ForkProcess-852:
Process ForkProcess-847:
Process ForkProcess-850:
Process ForkProcess-845:
Process ForkProcess-851:
Process ForkProcess-843:
Process ForkProcess-846:
Process ForkProcess-844:
Process ForkProcess-842:
Process ForkProcess-841:
Process ForkProcess-848:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108,

KeyboardInterrupt: 

2465